# Dataset with holidays data [COVID-19: Holidays of countries](https://www.kaggle.com/vbmokin/covid19-holidays-of-countries)

# COVID-19: EDA & Forecasting with holidays impact for confirmed cases. Prophet with many parameters optimization. Forecasting for 2 future weeks. Using very long lasting seasonality.

# Acknowledgements

- dataset [COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19)
- my dataset with holidays data [COVID-19: Holidays of countries](https://www.kaggle.com/vbmokin/covid19-holidays-of-countries) - it is recommended to follow the updates
- and others datasets
- notebook with the code to read the data [COVID-19: current situation on August](https://www.kaggle.com/corochann/covid-19-current-situation-on-august)
- notebook [COVID-19 Novel Coronavirus EDA & Forecasting Cases](https://www.kaggle.com/khoongweihao/covid-19-novel-coronavirus-eda-forecasting-cases) from [@Wei Hao Khoong]
(https://www.kaggle.com/khoongweihao)
- notebook with code for parameters tuning [COVID-19-in-Ukraine: Prophet & holidays tuning](https://www.kaggle.com/vbmokin/covid-19-in-ukraine-prophet-holidays-tuning)
- https://facebook.github.io/prophet/
- https://facebook.github.io/prophet/docs/
- https://github.com/facebook/prophet
- https://github.com/dr-prodigy/python-holidays

There are many studies in the field of coronavirus forecasting. Many researchers use **Prophet** (from Facebook). But for some reason, no one takes into account the holidays impact. After all, despite all the prohibitions, it is difficult for people to stay at home and they still somehow celebrate the **holidays** to which they are accustomed. The desire to celebrate is especially strong when people are sitting at home all the time looking for something to do. In our opinion, the impact of the holidays is manifested in the fact that within 4-10 days after these holidays there may be a jump in the number of confirmed cases, due to the fact that people went shopping, and even visiting each other, perhaps even in violation of quarantine requirements. 

The Prophet uses the library [holidays](https://github.com/dr-prodigy/python-holidays) with information about the main holidays of 67 countries, but and its package has some disadvantages. That's why I created a more perfect own dataset and plan to update it periodically. My graduate students ( ) help me fill it. Now my dataset [COVID-19: Holidays of countries](https://www.kaggle.com/vbmokin/covid19-holidays-of-countries) has holidays for 70 countries and more adapted for use in the prediction of coronavirus diseases. 

We will limit myself to forecasting only those countries for which there is information about holidays for Prophet.

A multiplicative Prophet model is built taking into account the weekly and very long lasting (4(N+7) days) seasonality, first without taking into account the holidays, and then taking into account the holidays. Then the model is tuned according to the parameter "prior_scale" that gets the value from the list of the user and is searching the one that provides the best forecast of the 14 last values of the data. The optimal model is determined for each country (with or without holidays, the best value of parameter "prior_scale" ). 

The model with optimal parameters is used to predict future data for the next 3 days. The data is taken from [COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19) (usually this dataset are updated there daily and are available as of yesterday), so the next 3 days are counted from the date of the last committee of this notebook. 

For the best models, plots with forecast values and with all components of the model are built.

There is a version of this notebook with model validation, there are versions for some countries, where the model is tuned simultaneously for all three parameters of the holidays (lower_window, upper_window, prior_scale) - see among [notebooks of the dataset **"COVID-19: Holidays of countries"**](https://www.kaggle.com/vbmokin/covid19-holidays-of-countries/notebooks)

The forecast for all countries with the tuning of only one parameter (prior_scale) gives low accuracy unfortunately. The purpose of this notebook is to show how to use a holiday [dataset](https://www.kaggle.com/vbmokin/covid19-holidays-of-countries) for 70 countries.

Higher forecasting accuracy (up to 2-6% for 7 days) is achieved by tuning 11 parameters for each country individually - see their list in the [dataset](https://www.kaggle.com/vbmokin/covid19-holidays-of-countries). For example, see notebook for Ukraine.

<a class="anchor" id="0.1"></a>
## Table of Contents

1. [Import libraries](#1)
1. [Download data](#2)
1. [Selection of countries with data on holidays](#3)
1. [EDA](#4)
1. [Model training, forecasting and evaluation](#5)


## 1. Import libraries<a class="anchor" id="1"></a>

[Back to Table of Contents](#0.1)

Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
import requests
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go

from datetime import date, timedelta, datetime
from fbprophet import Prophet
from fbprophet.make_holidays import make_holidays_df
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric
import holidays

import pycountry
import plotly.express as px
from collections import namedtuple

import warnings
warnings.simplefilter('ignore')

## 2. Download data<a class="anchor" id="2"></a>

[Back to Table of Contents](#0.1)

In [2]:
# Thanks https://github.com/CSSEGISandData/COVID-19
filename = 'time_series_covid19_confirmed_global.csv'
print(f'Downloading {filename}')
myfile = requests.get(f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/{filename}')
open('data', 'wb').write(myfile.content)
confirmed_global_df = pd.read_csv('data')
confirmed_global_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,703228,703228,703228,703228,703228,703228,703228,703228,703228,703228
285,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
286,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,11945,11945,11945,11945,11945,11945,11945,11945,11945,11945
287,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,343012,343012,343079,343079,343079,343135,343135,343135,343135,343135


In [3]:
# Thanks to https://www.kaggle.com/corochann/covid-19-current-situation-on-august
def _convert_date_str(df):
    try:
        df.columns = list(df.columns[:4]) + [datetime.strptime(d, "%m/%d/%y").date().strftime("%Y-%m-%d") for d in df.columns[4:]]
    except:
        print('_convert_date_str failed with %y, try %Y')
        df.columns = list(df.columns[:4]) + [datetime.strptime(d, "%m/%d/%Y").date().strftime("%Y-%m-%d") for d in df.columns[4:]]

_convert_date_str(confirmed_global_df)
confirmed_global_df

,Province/State,Country/Region,Lat,Long,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2023-02-28,2023-03-01,2023-03-02,2023-03-03,2023-03-04,2023-03-05,2023-03-06,2023-03-07,2023-03-08,2023-03-09
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,703228,703228,703228,703228,703228,703228,703228,703228,703228,703228
285,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
286,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,11945,11945,11945,11945,11945,11945,11945,11945,11945,11945
287,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,343012,343012,343079,343079,343079,343135,343135,343135,343135,343135


In [4]:
# Thanks to https://www.kaggle.com/corochann/covid-19-current-situation-on-august
df = confirmed_global_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='ConfirmedCases')

In [5]:
df

,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0
...,...,...,...,...,...,...
330322,NaN,West Bank and Gaza,31.952200,35.233200,2023-03-09,703228
330323,NaN,Winter Olympics 2022,39.904200,116.407400,2023-03-09,535
330324,NaN,Yemen,15.552727,48.516388,2023-03-09,11945
330325,NaN,Zambia,-13.133897,27.849332,2023-03-09,343135


In [6]:
df["Country/Region"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Gua

It would be more correct to call this list "Country/Regions"

In [7]:
# Convert name of countries to ISO 3166
df["Country/Region"].replace({'Korea, South': 'Korea, Republic of'}, inplace=True)
df["Country/Region"].replace({'Russia': 'Russian Federation'}, inplace=True)
df["Country/Region"].replace({'US': 'United States'}, inplace=True)

In [8]:
df

,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0
...,...,...,...,...,...,...
330322,NaN,West Bank and Gaza,31.952200,35.233200,2023-03-09,703228
330323,NaN,Winter Olympics 2022,39.904200,116.407400,2023-03-09,535
330324,NaN,Yemen,15.552727,48.516388,2023-03-09,11945
330325,NaN,Zambia,-13.133897,27.849332,2023-03-09,343135


In [9]:
df2 = df.groupby(["Date", "Country/Region"])[['Date', 'Country/Region', 'ConfirmedCases']].sum().reset_index()

In [10]:
df2_last_4days = df2[df2['Date'] > ((datetime.strptime(df2['Date'].max(), "%Y-%m-%d")-timedelta(days = 4)).strftime("%Y-%m-%d"))]
df2_last_4days

,Date,Country/Region,ConfirmedCases
228939,2023-03-06,Afghanistan,209406
228940,2023-03-06,Albania,334427
228941,2023-03-06,Algeria,271477
228942,2023-03-06,Andorra,47875
228943,2023-03-06,Angola,105277
...,...,...,...
229738,2023-03-09,West Bank and Gaza,703228
229739,2023-03-09,Winter Olympics 2022,535
229740,2023-03-09,Yemen,11945
229741,2023-03-09,Zambia,343135


In [11]:
df_countries = df2_last_4days['Country/Region'].unique()
df_countries

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Gua

In [12]:
latest_date = df2['Date'].max()
latest_date

'2023-03-09'

In [13]:
# Quality of data for each country
N = len(df2[df2['Country/Region'] == 'United States'])
N

1143

## 3. Selection of countries with data on holidays<a class="anchor" id="3"></a>

[Back to Table of Contents](#0.1)

### Thank to dataset [COVID-19: Holidays of countries](https://www.kaggle.com/vbmokin/covid19-holidays-of-countries)

In [14]:
# Thanks to dataset https://www.kaggle.com/vbmokin/covid19-holidays-of-countries
holidays_df = pd.read_csv('../input/covid19-holidays-of-countries/holidays_df_of_70_countries_for_covid_19_2021.csv')
holidays_df

,ds_holidays,holiday,ds,country,code,country_official_name,lower_window,upper_window,prior_scale,source
0,2020-02-24,Día de Carnaval [Carnival's Day],2020-03-02,Argentina,AR,Argentine Republic,-3,3,10,https://github.com/dr-prodigy/python-holidays
1,2020-02-25,Día de Carnaval [Carnival's Day],2020-03-03,Argentina,AR,Argentine Republic,-3,3,10,https://github.com/dr-prodigy/python-holidays
2,2020-03-24,Día Nacional de la Memoria por la Verdad y la ...,2020-03-31,Argentina,AR,Argentine Republic,-3,3,10,https://github.com/dr-prodigy/python-holidays
3,2020-04-09,Semana Santa (Jueves Santo) [Holy day (Holy T...,2020-04-16,Argentina,AR,Argentine Republic,-3,3,10,https://github.com/dr-prodigy/python-holidays
4,2020-04-10,Semana Santa (Viernes Santo) [Holy day (Holy ...,2020-04-17,Argentina,AR,Argentine Republic,-3,3,10,https://github.com/dr-prodigy/python-holidays
...,...,...,...,...,...,...,...,...,...,...
1718,2021-05-09,Victory Day and Commemoration of the Fallen He...,2021-05-16,Moldova,MD,Republic of Moldova,-3,3,10,https://en.wikipedia.org/wiki/Public_holidays_...
1719,2021-06-01,Children's Day,2021-06-08,Moldova,MD,Republic of Moldova,-3,3,10,https://en.wikipedia.org/wiki/Public_holidays_...
1720,2021-08-27,Independence Day,2021-09-03,Moldova,MD,Republic of Moldova,-3,3,10,https://en.wikipedia.org/wiki/Public_holidays_...
1721,2021-08-31,National Language Day,2021-09-07,Moldova,MD,Republic of Moldova,-3,3,10,https://en.wikipedia.org/wiki/Public_holidays_...


In [15]:
holidays_df['country'].unique()

array(['Argentina', 'Austria', 'Australia', 'Bangladesh', 'Belgium',
       'Bulgaria', 'Burundi', 'Brazil', 'Belarus', 'Canada',
       'Switzerland', 'Chile', 'China', 'Colombia', 'Czechia', 'Germany',
       'Denmark', 'Dominican Republic', 'Estonia', 'Egypt', 'Spain',
       'Finland', 'France', 'United Kingdom', 'Greece', 'Honduras',
       'Croatia', 'Hungary', 'Indonesia', 'Ireland', 'Israel', 'India',
       'Iceland', 'Italy', 'Japan', 'Kenya', 'Korea, Republic of',
       'Lithuania', 'Luxembourg', 'Latvia', 'Morocco', 'Mexico',
       'Malaysia', 'Nigeria', 'Nicaragua', 'Netherlands', 'Norway',
       'New Zealand', 'Peru', 'Philippines', 'Pakistan', 'Poland',
       'Portugal', 'Paraguay', 'Romania', 'Serbia', 'Russian Federation',
       'Sweden', 'Singapore', 'Slovenia', 'Slovakia', 'Thailand',
       'Turkey', 'Ukraine', 'United States', 'Viet Nam', 'South Africa',
       'Georgia', 'Albania', 'Moldova'], dtype=object)

In [ ]:
holidays_df_code_countries = holidays_df['code'].unique()
holidays_df_code_countries

In [ ]:
def dict_code_countries_with_holidays(list_name_countries: list,
                                      holidays_df: pd.DataFrame()):
    """
    Defines a dictionary with the names of user countries and their two-letter codes (ISO 3166) 
    in the dataset "COVID-19: Holidays of countries" 
    
    Returns: 
    - countries: dictionary with the names of user countries and their two-letter codes (ISO 3166) 
    - holidays_df_identificated: DataFrame with holidays data for countries from dictionary 'countries'
    
    Args: 
    - list_name_countries: list of the name of countries (name or common_name or official_name or alha2 or alpha3 codes from ISO 3166)
    - holidays_df: DataFrame with holidays "COVID-19: Holidays of countries"
    """
    
    import pycountry
    
    # Identification of countries for which there are names according to ISO
    countries = {}
    dataset_all_countries = list(holidays_df['code'].unique())
    list_name_countries_identificated = []
    list_name_countries_not_identificated = []
    for country in list_name_countries:
        try: 
            country_id = pycountry.countries.get(alpha_2=country)
            if country_id.alpha_2 in dataset_all_countries:
                countries[country] = country_id.alpha_2
        except AttributeError:
            try: 
                country_id = pycountry.countries.get(name=country)
                if country_id.alpha_2 in dataset_all_countries:
                    countries[country] = country_id.alpha_2
            except AttributeError:
                try: 
                    country_id = pycountry.countries.get(official_name=country)
                    if country_id.alpha_2 in dataset_all_countries:
                        countries[country] = country_id.alpha_2
                except AttributeError:
                    try: 
                        country_id = pycountry.countries.get(common_name=country)
                        if country_id.alpha_2 in dataset_all_countries:
                            countries[country] = country_id.alpha_2
                    except AttributeError:
                        try: 
                            country_id = pycountry.countries.get(alpha_3=country)
                            if country_id.alpha_2 in dataset_all_countries:
                                countries[country] = country_id.alpha_2
                        except AttributeError:
                            list_name_countries_not_identificated.append(country)
    holidays_df_identificated = holidays_df[holidays_df['code'].isin(countries.values())]
    
    print(f'Thus, the dataset has holidays in {len(countries)} countries from your list with {len(list_name_countries)} countries')
    if len(countries) == len(dataset_all_countries):
        print('All available in this dataset holiday data is used')
    else:
        print("Holidays are available in the dataset for such countries (if there are countries from your list, then it's recommended making changes to the list)")
        print(np.array(holidays_df[~holidays_df['code'].isin(countries.values())].country_official_name.unique()))
        
    return countries, holidays_df_identificated

In [ ]:
countries_dict, holidays_df = dict_code_countries_with_holidays(df_countries, holidays_df)

In [ ]:
def adaption_df_to_holidays_df_for_prophet(df, col, countries_dict):
    # Adaptation the dataframe df (by column=col) to holidays_df by list of countries in dictionary countries_dict
    
    # Filter df for countries which there are in the dataset with holidays
    df = df[df[col].isin(list(countries_dict.keys()))].reset_index(drop=True)
    
    # Add alpha_2 (code from ISO 3166) for each country
    df['iso_alpha'] = None
    for key, value in countries_dict.items():
        df.loc[df[col] == key, 'iso_alpha'] = value    
    
    return df

In [ ]:
holidays_df

In [ ]:
df2 = adaption_df_to_holidays_df_for_prophet(df2, 'Country/Region', countries_dict)
df2.columns = ['Date', 'Country', 'Confirmed', 'iso_alpha']
df2

In [ ]:
print("Number of countries/regions with data: " + str(len(df2.Country.unique())))

## 4. EDA<a class="anchor" id="4"></a>

[Back to Table of Contents](#0.1)

## Earliest Cases

In [23]:
df2.describe()

,Confirmed
count,8.001000e+04
mean,3.624703e+06
std,9.408914e+06
min,0.000000e+00
25%,7.442525e+04
50%,6.322200e+05
75%,2.933504e+06
max,1.038027e+08


In [24]:
df2.head()

,Date,Country,Confirmed,iso_alpha
0,2020-01-22,Albania,0,AL
1,2020-01-22,Argentina,0,AR
2,2020-01-22,Australia,0,AU
3,2020-01-22,Austria,0,AT
4,2020-01-22,Bangladesh,0,BD


## Latest Cases

In [25]:
df2.tail()

,Date,Country,Confirmed,iso_alpha
80005,2023-03-09,Turkey,17042722,TR
80006,2023-03-09,Ukraine,5711929,UA
80007,2023-03-09,United Kingdom,24658705,GB
80008,2023-03-09,United States,103802702,US
80009,2023-03-09,Vietnam,11526994,VN


## 5. Model training, forecasting and evaluation<a class="anchor" id="5"></a>

[Back to Table of Contents](#0.1)

Forecasting Confirmed Cases Worldwide with Prophet by Country

In [26]:
lower_window_list = [0, -1, -2, -3] # must be exactly 4 values (identical allowed)
upper_window_list = [0, 1, 2, 3] # must be exactly 4 values (identical allowed)
prior_scale_list = [0.05, 0.5, 1, 15] # must be exactly 4 values (identical allowed)

In [27]:
# Set very long lasting period of seasonality
T = 4*(N+7)
T

4600

In [28]:
def convert10_base4(n):
    # convert decimal to base 4
    alphabet = "0123"
    if n < 4:
        return alphabet[n]
    else:
        return (convert10_base4(n // 4) + alphabet[n % 4]).format('4f')

In [29]:
days_to_forecast = 7 # in future (after training data)
days_to_forecast_for_evalution = 7 # on the latest training data - for model training
first_forecasted_date = sorted(list(set(df2['Date'].values)))[-days_to_forecast]

print('The first date to perform forecasts for evaluation is: ' + str(first_forecasted_date))

The first date to perform forecasts for evaluation is: 2023-03-03


In [ ]:
print('The end date to perform forecasts in future for is: ' + (datetime.strptime(df2['Date'].max(), "%Y-%m-%d")+timedelta(days = days_to_forecast)).strftime("%Y-%m-%d"))

In [ ]:
confirmed_df = df2[['Date', 'Country', 'Confirmed', 'iso_alpha']]
confirmed_df

In [32]:
all_countries = confirmed_df['Country'].unique()
all_countries

array(['Albania', 'Argentina', 'Australia', 'Austria', 'Bangladesh',
       'Belarus', 'Belgium', 'Brazil', 'Bulgaria', 'Burundi', 'Canada',
       'Chile', 'China', 'Colombia', 'Croatia', 'Czechia', 'Denmark',
       'Dominican Republic', 'Egypt', 'Estonia', 'Finland', 'France',
       'Georgia', 'Germany', 'Greece', 'Honduras', 'Hungary', 'Iceland',
       'India', 'Indonesia', 'Ireland', 'Israel', 'Italy', 'Japan',
       'Kenya', 'Korea, Republic of', 'Latvia', 'Lithuania', 'Luxembourg',
       'Malaysia', 'Mexico', 'Moldova', 'Morocco', 'Netherlands',
       'New Zealand', 'Nicaragua', 'Nigeria', 'Norway', 'Pakistan',
       'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Romania',
       'Russian Federation', 'Serbia', 'Singapore', 'Slovakia',
       'Slovenia', 'South Africa', 'Spain', 'Sweden', 'Switzerland',
       'Thailand', 'Turkey', 'Ukraine', 'United Kingdom', 'United States',
       'Vietnam'], dtype=object)

In [33]:
n = 64 # number of combination of parameters lower_window / upper_window / prior_scale

In [34]:
def make_forecasts(all_countries, confirmed_df, holidays_df, days_to_forecast, days_to_forecast_for_evalution, first_forecasted_date):
    # Thanks to https://www.kaggle.com/vbmokin/covid-19-in-ukraine-prophet-holidays-tuning
    
    def eval_error(forecast_df, country_df_val, first_forecasted_date, title):
        # Evaluate forecasts with validation set val_df and calculaction and printing with title the relative error
        forecast_df[forecast_df['yhat'] < 0]['yhat'] = 0
        result_df = forecast_df[(forecast_df['ds'] >= pd.to_datetime(first_forecasted_date))]
        result_val_df = result_df.merge(country_df_val, on=['ds'])
        result_val_df['rel_diff'] = (result_val_df['y'] - result_val_df['yhat']).round().abs()
        relative_error = [sum(result_val_df['rel_diff'].values)*100/result_val_df['y'].sum()]
        return relative_error
    
    def model_training_forecasting(df, forecast_days, holidays_df=None):
        # Prophet model training and forecasting
        
        model = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False, 
                        holidays=holidays_df, changepoint_range=1, changepoint_prior_scale = 0.05)
        model.add_seasonality(name='weekly', period=7, fourier_order=8, mode = 'multiplicative', prior_scale = 0.2)
        model.add_seasonality(name='many_days', period=T, fourier_order=6, mode = 'multiplicative', prior_scale = 0.2)
        model.fit(df)
        future = model.make_future_dataframe(periods=forecast_days)
        forecast = model.predict(future)
        
        # Make values integer, and replace negative values with zero
        feature_all = ['yhat_lower', 'yhat', 'yhat_upper']
        forecast[feature_all] = forecast[feature_all].round().astype('int')
        for feature in feature_all:
            forecast.loc[forecast[feature] < 0, feature] = 0
    
        return model, forecast
    
    forecast_dfs = []
    relative_errors = []
    cols_w = ['ds', 'trend', 'yhat', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper', 'additive_terms', 'additive_terms_lower', 'additive_terms_upper',
              'multiplicative_terms','multiplicative_terms_lower', 'multiplicative_terms_upper', 'weekly', 'weekly_lower', 'weekly_upper']
    cols_h = ['ds', 'trend', 'yhat', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper', 'additive_terms', 'additive_terms_lower', 'additive_terms_upper',
              'holidays', 'holidays_lower', 'holidays_upper', 'multiplicative_terms','multiplicative_terms_lower', 'multiplicative_terms_upper', 'weekly',
              'weekly_lower', 'weekly_upper']
    relative_errors_holidays = []
    counter = 0
    results = pd.DataFrame(columns=['Country', 'Code', 'Real', 'Pred', 'Pred_h', 'n_h', 'err', 'err_h', 'lower_w', 'upper_w', 'prior_sc', 'less, %'])
    
    
    for j in range(len(all_countries)):
        country = all_countries[j]
        if country in confirmed_df['Country'].values:
            print(f'Country {str(country)} is listed')
            country_df = confirmed_df[confirmed_df['Country'] == country].reset_index(drop=True)
            country_iso_alpha = country_df['iso_alpha'][0]
            
            # Calc daily values and ignoring zero daily total cases
            country_df['Confirmed'] = country_df['Confirmed'].diff().reset_index(drop=True)
            country_df.loc[0,'Confirmed'] = 0
            country_df = country_df[country_df['Confirmed'] > 0].reset_index(drop=True)
            first_forecasted_date = sorted(list(set(country_df['Date'].values)))[-days_to_forecast]
            
            # Selection holidays of country
            country_holidays_df = holidays_df[holidays_df['code'] == country_iso_alpha][['ds', 'holiday', 'lower_window', 'upper_window', 'prior_scale']].reset_index(drop=True)
            country_dfs = []            
            
            # Data preparation for forecast with Prophet
            country_df = country_df[['Date', 'Confirmed']]
            country_df.columns = ['ds','y']
            country_df['ds'] = pd.to_datetime(country_df['ds'])

            # Set training and validation datasets
            country_df_future = country_df.copy()
            country_df_val = country_df[(country_df['ds'] >= pd.to_datetime(first_forecasted_date))].copy()
            country_df = country_df[(country_df['ds'] < pd.to_datetime(first_forecasted_date))]

            # Without holidays
            # Model training and forecasting without holidays
            model, forecast = model_training_forecasting(country_df, days_to_forecast_for_evalution)
            #fig1 = model.plot_components(forecast)

            # Evaluate forecasts with validation set val_df and calculaction and printing the relative error
            forecast_df = forecast[['ds', 'yhat']].copy()
            relative_errors += eval_error(forecast_df, country_df_val, first_forecasted_date, 'without holidays')

            # With holidays
            # Model training with tuning prior_scale and forecasting
            relative_error_holidays_min = relative_errors[-1]
            number_holidays = len(country_holidays_df[(country_holidays_df['ds'] > '2020-01-21') & (country_holidays_df['ds'] < '2021-06-01')])
            for i in range(n):
                parameters_iter = convert10_base4(i).zfill(3)
                lower_window_i = lower_window_list[int(parameters_iter[0])]
                upper_window_i = upper_window_list[int(parameters_iter[1])]
                prior_scale_i = prior_scale_list[int(parameters_iter[2])]
                country_holidays_df['lower_window'] = lower_window_i
                country_holidays_df['upper_window'] = upper_window_i
                country_holidays_df['prior_scale'] = prior_scale_i
                model_holidays, forecast_holidays = model_training_forecasting(country_df, days_to_forecast_for_evalution, country_holidays_df)
                
                # Evaluate forecasts with validation set val_df and calculaction and printing the relative error
                forecast_holidays_df = forecast_holidays[['ds', 'yhat']].copy()
                relative_error_holidays = eval_error(forecast_holidays_df, country_df_val, first_forecasted_date, 'with holidays impact')
                
                # Save results
                if i == 0:
                    relative_error_holidays_min = relative_error_holidays
                    forecast_holidays_df_best = forecast_holidays[cols_h]
                    model_holidays_best = model_holidays
                    lower_window_best = lower_window_i
                    upper_window_best = upper_window_i
                    prior_scale_best = prior_scale_i
                elif (relative_error_holidays[0] < relative_error_holidays_min[0]):
                    relative_error_holidays_min = relative_error_holidays
                    forecast_holidays_df_best = forecast_holidays[cols_h]
                    model_holidays_best = model_holidays
                    lower_window_best = lower_window_i
                    upper_window_best = upper_window_i
                    prior_scale_best = prior_scale_i
                print('i =',i,' from',n,':  lower_window =', lower_window_i, 'upper_window =',upper_window_i, 'prior_scale =', prior_scale_i)
                print('error_holidays =',relative_error_holidays[0], 'err_holidays_min (WAPE)',relative_error_holidays_min[0], '\n')
            
            # Results visualization
            print('The best errors of model with holidays is', relative_error_holidays_min[0], 'with lower_window =', str(lower_window_best),
              ' upper_window =', str(upper_window_best), ' prior_scale =', str(prior_scale_best))
            print('The best errors WAPE of model with holidays is', relative_error_holidays_min[0], '\n')
            relative_errors_holidays += relative_error_holidays_min            

            # Save results to dataframe with all dates
            forecast_holidays_df_best['country'] = country
            forecast_holidays_df_best.rename(columns={'yhat':'confirmed'}, inplace=True)
            if j == 0:                
                forecast_holidays_dfs = forecast_holidays_df_best.tail(days_to_forecast_for_evalution)
            else:
                forecast_holidays_dfs = pd.concat([forecast_future_dfs, forecast_holidays_df_best.tail(days_to_forecast_for_evalution)])

            # Forecasting the future
            if relative_errors[-1] < relative_errors_holidays[-1]:
                # The forecast without taking into account the holidays is the best
                model_future_best, forecast_future_best = model_training_forecasting(country_df_future, days_to_forecast)
                forecast_plot = model_future_best.plot(forecast_future_best, ylabel='Confirmed in '+ country + ' (forecasting without holidays)')
                cols = cols_w
            else:
                # The forecast taking into account the holidays is the best
                country_holidays_df['prior_scale'] = prior_scale_best
                model_future_best, forecast_future_best = model_training_forecasting(country_df_future, days_to_forecast, country_holidays_df)
                forecast_plot = model_future_best.plot(forecast_future_best, ylabel='Confirmed in '+ country + ' (forecasting with holidays)')
                cols = cols_h
            # Save forecasting results 
            forecast_future_df_best = forecast_future_best[cols]
            forecast_future_df_best['country'] = country
            forecast_future_df_best.rename(columns={'yhat':'confirmed'}, inplace=True)
            if j == 0:                
                forecast_future_dfs = forecast_future_df_best.tail(days_to_forecast)
            else:
                forecast_future_dfs = pd.concat([forecast_future_dfs, forecast_future_df_best.tail(days_to_forecast)])
            
            # Save results to dataframe with result for the last date
            results.loc[j,'Country'] = country
            results.loc[j,'Code'] = country_iso_alpha
            confirmed_real_last = country_df_val.tail(1)['y'].values[0].astype('int')
            results.loc[j,'Real'] = confirmed_real_last if confirmed_real_last > 0 else 0
            confirmed_pred_last = round(forecast_df.tail(1)['yhat'].values[0]).astype('int')
            results.loc[j,'Pred'] = confirmed_pred_last if confirmed_pred_last > 0 else 0
            confirmed_pred_holidays_last = round(forecast_holidays_df_best.tail(1)['confirmed'].values[0],0).astype('int')
            results.loc[j,'Pred_h'] = confirmed_pred_holidays_last if confirmed_pred_holidays_last > 0 else 0
            results.loc[j,'n_h'] = number_holidays
            results.loc[j,'err'] = relative_errors[-1]
            results.loc[j,'err_h'] = relative_errors_holidays[-1]
            results.loc[j,'lower_w'] = lower_window_best
            results.loc[j,'upper_w'] = upper_window_best
            results.loc[j,'prior_sc'] = prior_scale_best
            results.loc[j,'less, %'] = round((relative_errors[-1]-relative_errors_holidays[-1])*100/relative_errors[-1],1)
            
            # Output forecasting data
            forecast_future_opt_future = forecast_future_df_best[['ds', 'yhat_lower', 'confirmed', 'yhat_upper']].copy().tail(days_to_forecast)
            forecast_future_opt_future.columns = ['ds', 'confirmed_lower', 'confirmed', 'confirmed_upper']
            display(forecast_future_opt_future)
            
            # Output plot
            model_future_best.plot_components(forecast_future_best)
            

        else:
            print('Country ' + str(country) + ' is not listed! ')
            continue
            
    return forecast_holidays_dfs, relative_errors_holidays, forecast_future_dfs, results

In [ ]:
forecast_holidays_dfs, relative_errors_holidays, forecast_future_dfs, results = make_forecasts(all_countries, confirmed_df, holidays_df, 
                                                                                               days_to_forecast, days_to_forecast_for_evalution, first_forecasted_date)

Country Albania is listed
i = 0  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 0.05
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 1  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 0.5
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 2  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 1
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 3  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 15
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 4  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 0.05
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 5  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 0.5
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 6  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 1
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 7  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 15
error_holidays = 100.0 err_holida

,ds,confirmed_lower,confirmed,confirmed_upper
1029,2023-03-10,0,0,321
1030,2023-03-11,0,0,323
1031,2023-03-12,0,0,326
1032,2023-03-13,0,0,355
1033,2023-03-14,0,0,329
1034,2023-03-15,0,0,329
1035,2023-03-16,0,0,331


Country Argentina is listed
i = 0  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 0.05
error_holidays = 296.48247039702807 err_holidays_min (WAPE) 296.48247039702807 

i = 1  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 0.5
error_holidays = 287.508706756443 err_holidays_min (WAPE) 287.508706756443 

i = 2  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 1
error_holidays = 274.62270722080336 err_holidays_min (WAPE) 274.62270722080336 

i = 3  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 15
error_holidays = 280.20664035291384 err_holidays_min (WAPE) 274.62270722080336 

i = 4  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 0.05
error_holidays = 285.17529602971905 err_holidays_min (WAPE) 274.62270722080336 

i = 5  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 0.5
error_holidays = 297.8523334107267 err_holidays_min (WAPE) 274.62270722080336 

i = 6  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 1
erro

,ds,confirmed_lower,confirmed,confirmed_upper
813,2023-03-07,789,22182,45387
814,2023-03-08,763,22977,46951
815,2023-03-09,444,22877,44822
816,2023-03-10,0,21439,44225
817,2023-03-11,0,13428,34854
818,2023-03-12,0,8453,31730
819,2023-03-13,0,15377,38056


Country Australia is listed
i = 0  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 0.05
error_holidays = 290.26849504494123 err_holidays_min (WAPE) 290.26849504494123 

i = 1  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 0.5
error_holidays = 289.4906660520857 err_holidays_min (WAPE) 289.4906660520857 

i = 2  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 1
error_holidays = 280.20857340401017 err_holidays_min (WAPE) 280.20857340401017 

i = 3  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 15
error_holidays = 287.5835445955289 err_holidays_min (WAPE) 280.20857340401017 

i = 4  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 0.05
error_holidays = 282.76100483982486 err_holidays_min (WAPE) 280.20857340401017 

i = 5  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 0.5
error_holidays = 294.1057847430283 err_holidays_min (WAPE) 280.20857340401017 

i = 6  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 1
err

...

Country Bangladesh is listed
i = 0  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 0.05
error_holidays = 724.0 err_holidays_min (WAPE) 724.0 

i = 1  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 0.5
error_holidays = 530.0 err_holidays_min (WAPE) 530.0 

i = 2  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 1
error_holidays = 556.0 err_holidays_min (WAPE) 530.0 

i = 3  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 15
error_holidays = 376.0 err_holidays_min (WAPE) 376.0 

i = 4  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 0.05
error_holidays = 620.0 err_holidays_min (WAPE) 376.0 

i = 5  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 0.5
error_holidays = 724.0 err_holidays_min (WAPE) 376.0 

i = 6  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 1
error_holidays = 632.0 err_holidays_min (WAPE) 376.0 

i = 7  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 15
error_holidays = 332.0 err_hol

,ds,confirmed_lower,confirmed,confirmed_upper
1055,2023-03-09,0,13,1656
1056,2023-03-10,0,10,1657
1057,2023-03-11,0,0,1709
1058,2023-03-12,0,6,1455
1059,2023-03-13,0,14,1552
1060,2023-03-14,0,13,1623
1061,2023-03-15,0,11,1651


Country Belarus is listed
i = 0  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 0.05
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 1  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 0.5
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 2  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 1
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 3  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 15
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 4  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 0.05
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 5  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 0.5
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 6  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 1
error_holidays = 100.0 err_holidays_min (WAPE) 100.0 

i = 7  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 15
error_holidays = 100.0 err_holida

,ds,confirmed_lower,confirmed,confirmed_upper
781,2022-07-06,0,0,909
782,2022-07-07,0,0,946
783,2022-07-08,0,0,1026
784,2022-07-09,0,0,883
785,2022-07-10,0,0,627
786,2022-07-11,0,0,0
787,2022-07-12,0,0,99


Country Belgium is listed
i = 0  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 0.05
error_holidays = 25.233281493001556 err_holidays_min (WAPE) 25.233281493001556 

i = 1  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 0.5
error_holidays = 22.511664074650078 err_holidays_min (WAPE) 22.511664074650078 

i = 2  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 1
error_holidays = 19.673405909797822 err_holidays_min (WAPE) 19.673405909797822 

i = 3  from 64 :  lower_window = 0 upper_window = 0 prior_scale = 15
error_holidays = 30.05443234836703 err_holidays_min (WAPE) 19.673405909797822 

i = 4  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 0.05
error_holidays = 32.19284603421462 err_holidays_min (WAPE) 19.673405909797822 

i = 5  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 0.5
error_holidays = 22.55054432348367 err_holidays_min (WAPE) 19.673405909797822 

i = 6  from 64 :  lower_window = 0 upper_window = 1 prior_scale = 1
erro

In [ ]:
forecast_future_dfs

In [ ]:
forecast_holidays_dfs

In [ ]:
pd.set_option('max_rows', 75)

In [ ]:
print('Forecasting results')
display(results.sort_values(by=['err_h'], ascending=True))

In [ ]:
df_h_impact = results[(results['less, %'] > 0) & (results['err_h'] <= 20)]
if len(df_h_impact) > 0:
    print('Countries with the impact of holidays and with the sufficiently adequate model over the last weeks')
    display(df_h_impact.sort_values(by=['less, %'], ascending=False))
    print('Number of these countries is', len(df_h_impact))

In [ ]:
df_h_non_impact = results[(results['less, %'] <= 0) & (results['err'] <= 20)]
if len(df_h_non_impact) > 0:
    print('Countries without the impact of holidays and with a sufficiently adequate model over the last weeks')
    display(df_h_non_impact.sort_values(by=['less, %'], ascending=False))
    print('Number of these countries is', len(df_h_non_impact))

In [ ]:
# df_neutral = results[(results['less, %'] <= 1)]
# if len(df_neutral) > 0:
#     print('Others countries')
#     display(df_neutral.sort_values(by=['less, %'], ascending=False))
#     print('Number of these countries is', len(df_neutral))

In [ ]:
forecast_holidays_dfs.to_csv('forecast_holidays_dfs.csv', index=False)
forecast_future_dfs.to_csv('forecast_future_dfs.csv', index=False)
results.to_csv('results.csv', index=False)

In [ ]:
results[['Country', 'Code', 'lower_w', 'upper_w', 'prior_sc', 'err', 'err_h']].to_csv('holidays_params.csv', index=False)

I hope you find this notebook useful and enjoyable.

Your comments and feedback are most welcome.

[Go to Top](#0)